In [2]:
import datetime
import xarray
import pandas as pd
import pytz
import numpy as np
from pvlib import irradiance
from tqdm.notebook import tqdm
import netCDF4 as nc

from config import dni_data_path, ghi_data_path, solar_positions_path,data_path

tz = pytz.timezone("Etc/GMT+4")

poa_irradiance_folder = data_path / 'poa-irradiance'
poa_irradiance_folder.mkdir(exist_ok=True)

ghi_dataset = xarray.open_mfdataset(str(ghi_data_path) + "/*.nc", combine="nested", concat_dim="time")
ghi_dataset = ghi_dataset.to_dataframe().reset_index()
ghi_dataset = ghi_dataset.set_index('time')
latitudes = pd.unique(ghi_dataset['lat'])
longitudes = pd.unique(ghi_dataset['lon'])

dni_dataset = xarray.open_mfdataset(str(dni_data_path) + "/*.nc", combine="nested", concat_dim="time")
dni_dataset = dni_dataset.to_dataframe().reset_index()
dni_dataset = dni_dataset.set_index('time')


def get_data_at_loc(data, lat, lon, data_name):
    select1 = data[data["lat"] == lat]
    select2 = select1[select1["lon"] == lon]
    series = pd.Series(select2.index.values, index=select2[data_name])
    series = pd.Series(series.index.values, index=series, name=data_name)
    return series






In [3]:
# Load solar positions

solar_positions = {}

for latitude in latitudes:
    for longitude in longitudes:
        print(f"Loading solar position data for {latitude}_{longitude}")
        df = pd.HDFStore(solar_positions_path / f"{round(latitude, 2)}_{round(longitude, 2)}.h5")['solar_position']
        solar_positions[(latitude, longitude)] = df

Loading solar position data for -20.75_57.099998474121094
Loading solar position data for -20.75_57.150001525878906
Loading solar position data for -20.75_57.20000076293945
Loading solar position data for -20.75_57.25
Loading solar position data for -20.75_57.29999923706055
Loading solar position data for -20.75_57.349998474121094
Loading solar position data for -20.75_57.400001525878906
Loading solar position data for -20.75_57.45000076293945
Loading solar position data for -20.75_57.5
Loading solar position data for -20.75_57.54999923706055
Loading solar position data for -20.75_57.599998474121094
Loading solar position data for -20.75_57.650001525878906
Loading solar position data for -20.75_57.70000076293945
Loading solar position data for -20.75_57.75
Loading solar position data for -20.75_57.79999923706055
Loading solar position data for -20.75_57.849998474121094
Loading solar position data for -20.75_57.900001525878906
Loading solar position data for -20.75_57.95000076293945
Loa

In [4]:
solar_positions

{(-20.75,
  57.099998474121094):                            apparent_zenith      zenith  apparent_elevation  \
 time                                                                         
 2019-01-01 00:00:00+00:00       109.806947  109.806947          -19.806947   
 2019-01-01 00:30:00+00:00       103.967312  103.967312          -13.967312   
 2019-01-01 01:00:00+00:00        97.896513   97.896513           -7.896513   
 2019-01-01 01:30:00+00:00        91.643343   91.643343           -1.643343   
 2019-01-01 02:00:00+00:00        85.078409   85.245162            4.921591   
 ...                                    ...         ...                 ...   
 2019-12-31 21:30:00+00:00       132.491480  132.491480          -42.491480   
 2019-12-31 22:00:00+00:00       129.218862  129.218862          -39.218862   
 2019-12-31 22:30:00+00:00       125.162909  125.162909          -35.162909   
 2019-12-31 23:00:00+00:00       120.484202  120.484202          -30.484202   
 2019-12-31 23:30:00

In [3]:


def get_poa_irradiance(latitude, longitude, azimuth, tilt):
    solar_position = solar_positions[(latitude, longitude)]

    dni_series = get_data_at_loc(dni_dataset, latitude, longitude, "SID").tz_localize(pytz.UTC)
    ghi_series = get_data_at_loc(ghi_dataset, latitude, longitude, "SIS").tz_localize(pytz.UTC)
    dhi_series = ghi_series - dni_series * np.cos(np.radians(solar_position['zenith']))
    POA_irradiance = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        dni=dni_series,
        ghi=ghi_series,
        dhi=dhi_series,
        # dhi=clearsky['dhi'],
        dni_extra=irradiance.get_extra_radiation(ghi_series.index),
        solar_zenith=solar_position['apparent_zenith'],
        solar_azimuth=solar_position['azimuth'],
        albedo=0.2,
        model="perez",
    )['poa_global']
    return POA_irradiance


def calculate_and_save_poa_irradiance(azimuth, tilt, latitudes, longitudes):
    file_path = poa_irradiance_folder / f"{tilt}_{azimuth}.nc"
    if file_path.is_file():
        print(f"Skipping {tilt}_{azimuth}")
        return
    dataset = xarray.open_mfdataset(str(dni_data_path) + '/*.nc', combine="nested", concat_dim="time")
    dataset.to_netcdf(str(file_path))
    dataset.close()
    dataset = nc.Dataset(str(file_path), "r+")
    dataset.renameVariable('SID', 'POA_GLOBAL_IRRADIANCE')
    dataset.variables['POA_GLOBAL_IRRADIANCE'].long_name = "Plane of Array Irradiance"
    dataset.variables['POA_GLOBAL_IRRADIANCE'].units = "W/m^2"
    dataset.variables['POA_GLOBAL_IRRADIANCE'].set_auto_mask(False)
    dataset.variables['POA_GLOBAL_IRRADIANCE'].set_auto_scale(False)
    dataset.CDI = "Climate Data Interface version 1.9.8 (http://mpimet.mpg.de/cdi)"
    dataset.Conventions = "CF-1.6"
    dataset.history = f'Created {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

    dataset.creator_name = "Appadoo Apoorva Srinivas"
    dataset.creator_email = "apoorvaappadoo@gmail.com"

    dataset.project = "Mauritius Solar Pannel Placement Analysis"
    dataset.creator_type = "person"
    dataset.title = f"Plane of Array Irradiance for Mauritius at tilt {tilt} and azimuth {azimuth} for the year 2019"
    dataset.publisher_name = "Appadoo Apoorva Srinivas"
    dataset.publisher_email = "apoorvaappadoo@gmail.com"

    dataset.publisher_type = "person"
    dataset.summary = f"This file contains the plane of array irradiance for Mauritius at tilt {tilt} and azimuth {azimuth} for the year 2019"
    dataset.keywords = "Plane of Array Irradiance, Mauritius, Solar Pannels, Tilt, Azimuth"
    dataset.keywords_vocabulary = "GCMD Science Keywords"
    dataset.time_coverage_start = "2019-01-01"
    dataset.time_coverage_end = "2019-12-31"
    dataset.time_coverage_duration = "P1Y"
    dataset.time_coverage_resolution = "P30M"
    dataset.date_created = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    dataset.product_version = "1.0"

    dataset.publisher_url = ""
    dataset.institution = ""
    dataset.references = ""
    dataset.source = ""
    dataset.CMSAF_nwp_data = ""
    dataset.creator_url = ""
    dataset.instrument = ""
    dataset.instrument_vocabulary = ""
    dataset.platform = ""
    dataset.platform_vocabulary = ""

    # remove attributes
    dataset.__delattr__('publisher_url')
    dataset.__delattr__('institution')
    dataset.__delattr__('references')
    dataset.__delattr__('source')
    dataset.__delattr__('CMSAF_nwp_data')
    dataset.__delattr__('creator_url')
    dataset.__delattr__('instrument')
    dataset.__delattr__('instrument_vocabulary')
    dataset.__delattr__('platform')
    dataset.__delattr__('platform_vocabulary')

    for i, latitude in tqdm(enumerate(latitudes), desc="longitudes", leave=False, total=len(latitudes)):
        for j, longitude in enumerate(longitudes):
            dataset.variables['POA_GLOBAL_IRRADIANCE'][:, i, j] = get_poa_irradiance(latitude, longitude,
                                                                                     azimuth, tilt)

    dataset.close()

    print(f"Saved {tilt}_{azimuth}")



In [ ]:
azimuth_range = range(-180, 190, 10)
tilt_range = range(0, 100, 10)
for azimuth in tqdm(azimuth_range, desc="Azimuth", leave=False, total=len(azimuth_range)):
    for tilt in tqdm(tilt_range, desc="Tilt", leave=False, total=len(tilt_range)):
        calculate_and_save_poa_irradiance(azimuth, tilt, latitudes, longitudes)
